In [8]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

# Define the S3 paths
train_data = 's3://diabetes-buc/diabetes.csv'
output_path = 's3://diabetes-buc/output/'

# Create the SageMaker SKLearn estimator
estimator = SKLearn(
    entry_point='train.py',
    framework_version='1.0-1',  
    py_version='py3',
    instance_type='ml.m5.large',
    role='Sage-access',
    s3_output_path=output_path,
    hyperparameters={},
)

# Start the training job
estimator.fit({'train': train_data})


INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-11-21-20-36-40-591


2024-11-21 20:36:42 Starting - Starting the training job...
2024-11-21 20:36:56 Starting - Preparing the instances for training...
2024-11-21 20:37:26 Downloading - Downloading input data...
2024-11-21 20:37:56 Downloading - Downloading the training image.....2024-11-21 20:38:57,526 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-21 20:38:57,530 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-21 20:38:57,534 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-11-21 20:38:57,552 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-21 20:38:57,807 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-21 20:38:57,811 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-11-21 20:38:57,832 sagemaker-training-toolkit INFO     No GPUs detected (normal if n

In [26]:
from sagemaker.sklearn.model import SKLearnModel
import json
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# Define the model
model = SKLearnModel(
    model_data='s3://sagemaker-us-west-2-851725523927/sagemaker-scikit-learn-2024-11-21-20-36-40-591/output/model.tar.gz',  # Replace with the S3 path of your trained model artifact
    role='arn:aws:iam::851725523927:role/Sage-access',  # Replace with the SageMaker execution role ARN
    entry_point='inference.py',  # The script used for training
    framework_version='1.0-1'  # Framework version compatible with your training job
)

# Deploy the model to an endpoint
predictor = model.deploy(
    instance_type='ml.m5.large',  # Choose an instance type for deployment
    initial_instance_count=1,  # Number of instances to deploy
    serializer=JSONSerializer(),  # Serialize the input data to JSON
    deserializer=JSONDeserializer()
)



INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-11-22-04-31-10-836
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-11-22-04-31-11-384
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-11-22-04-31-11-384


------!

In [29]:
# Test the endpoint
sample_input = {
    "age": 45,
    "bmi": 25.6,
    "HbA1c_level": 5.8,
    "blood_glucose_level": 130,
    "hypertension": 0,
    "heart_disease": 0,
    "gender": "Male",
    "smoking_history": "non-smoker"
}


response = predictor.predict(sample_input)
print("Prediction:", response)

Prediction: {'predictions': [0]}


In [28]:
endpoint_name = predictor.endpoint_name
print(endpoint_name)


sagemaker-scikit-learn-2024-11-22-04-31-11-384
